In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))



In [1]:
%pip install google-cloud-bigquery
%pip install pandas
%pip install db-dtypes
%pip install pandas-gbq


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:

from google.cloud import bigquery
import pandas as pd

from utils.import_table import import_table
from utils.export import export_table


In [3]:
# Initialiser le client BigQuery
client = bigquery.Client.from_service_account_json('key.json')

In [4]:

locations = import_table(client, 'Location')
warehouses = import_table(client, 'Warehouse')
stores = import_table(client, 'Store')
suppliers = import_table(client, 'Supplier')

# Visualiser le DataFrame sous forme de tableau
display(locations.head())
display(warehouses.head())
display(stores.head())
display(suppliers.head())

/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id,capacity,address
0,86,5888,40901 Hall Lakes Apt. 934
1,63,9483,41499 Jones Pass
2,21,5902,941 Michelle Grove Apt. 552
3,30,9745,78193 William Drive
4,26,1300,622 Cheyenne Divide


,id,location_id,name
0,1,85,96501 Suzanne Village
1,2,57,84075 Maureen Alley Apt. 830
2,3,78,9411 Smith Radial


,id,name,location_id
0,1,80784 Joseph Landing Suite 415,1
1,2,161 Emily Wells Apt. 737,77
2,3,70936 Haney Causeway Suite 822,35
3,4,9570 Hill Valleys,22
4,5,7119 Salinas Court Suite 498,6


,id,entity_id,location_id,name
0,6,6,12,"Pitts, Lee and Fernandez"
1,11,11,13,Briggs-Wong
2,15,15,16,Green-Smith
3,12,12,23,"Mccall, Woodward and Holloway"
4,9,9,28,"Williams, Obrien and Marshall"


In [5]:
# Join des tables Locations et Warehouses sur id (locations) location_id (warehouses)
locations_warehouses = pd.merge(locations, warehouses, left_on='id', right_on='location_id')
locations_warehouses['type'] = 'warehouse'
locations_warehouses = locations_warehouses.drop(columns=['id_y', 'location_id'])
locations_warehouses = locations_warehouses.rename(columns={'id_x': 'id'})
display(locations_warehouses.head())

locations_stores = pd.merge(locations, stores, left_on='id', right_on='location_id')
locations_stores['type'] = 'store'
locations_stores = locations_stores.drop(columns=['id_y', 'location_id'])
locations_stores = locations_stores.rename(columns={'id_x': 'id'})
display(locations_stores.head())

locations_suppliers = pd.merge(locations, suppliers, left_on='id', right_on='location_id')
locations_suppliers['type'] = 'supplier'
locations_suppliers = locations_suppliers.drop(columns=['id_y', 'location_id', 'entity_id'])
locations_suppliers = locations_suppliers.rename(columns={'id_x': 'id'})
display(locations_suppliers.head())

,id,capacity,address,name,type
0,85,6006,914 Daniel Club Suite 848,96501 Suzanne Village,warehouse
1,78,3761,057 Javier Views,9411 Smith Radial,warehouse
2,57,7406,8992 Robin Island,84075 Maureen Alley Apt. 830,warehouse


,id,capacity,address,name,type
0,35,4121,57938 Koch Land,70936 Haney Causeway Suite 822,store
1,1,1060,8168 Lewis Ports Suite 707,80784 Joseph Landing Suite 415,store
2,39,1583,9873 Kathleen Parkways,778 Lang Cove Suite 536,store
3,22,2879,5290 John Ford Apt. 178,9570 Hill Valleys,store
4,6,3151,06121 Sandra Haven Suite 675,7119 Salinas Court Suite 498,store


,id,capacity,address,name,type
0,86,5888,40901 Hall Lakes Apt. 934,Carpenter PLC,supplier
1,28,1308,344 Paul Land Suite 688,"Williams, Obrien and Marshall",supplier
2,12,9252,448 Ryan Stravenue Apt. 743,"Pitts, Lee and Fernandez",supplier
3,56,2111,17451 Kevin Viaduct Suite 562,Fernandez-Wiley,supplier
4,92,8268,437 Graham Club Apt. 879,"Horton, Ramirez and Clark",supplier


In [6]:
# concaténation des tables
locations_warehouses_stores = pd.concat([locations_warehouses, locations_stores])
locations_warehouses_stores_suppliers = pd.concat([locations_warehouses_stores, locations_suppliers])

display(locations_warehouses_stores_suppliers.head())

,id,capacity,address,name,type
0,85,6006,914 Daniel Club Suite 848,96501 Suzanne Village,warehouse
1,78,3761,057 Javier Views,9411 Smith Radial,warehouse
2,57,7406,8992 Robin Island,84075 Maureen Alley Apt. 830,warehouse
0,35,4121,57938 Koch Land,70936 Haney Causeway Suite 822,store
1,1,1060,8168 Lewis Ports Suite 707,80784 Joseph Landing Suite 415,store


In [7]:
# Variables pour BigQuery
dataset_name = "warehouse_test"
table_name = "location"

export_table(client, locations_warehouses_stores_suppliers, dataset_name, table_name)

Dataset warehouse_test déjà existant.
